In [ ]:
import pandas as pd
import glob

files = sorted(glob.glob('run_ca*.csv'))
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

print(f"Loaded {len(files)} files, total rows = {len(df)}")
df.head()


In [ ]:
# Compute per-episode metrics
episode_stats = []
for (_, rad, ep), grp in df.groupby(['run_id', 'neighborhood_radius', 'episode_index']):
    steps = len(grp)
    mean_time = grp['time_delta_ms'].mean()
    actions = grp.sort_values('step_count')['action_taken'].tolist()
    if len(actions)>1:
        switches = sum(1 for i in range(1,len(actions)) if actions[i]!=actions[i-1])
        switch_rate = switches/(len(actions)-1)
    else:
        switch_rate = 0.0

    episode_stats.append({
        'radius':      rad,
        'steps':       steps,
        'time_ms':     mean_time,
        'switch_rate': switch_rate
    })

eps_df = pd.DataFrame(episode_stats)

# Aggregate by neighborhood radius
summary = eps_df.groupby('radius').agg(
    mean_steps   = ('steps',       'mean'),
    std_steps    = ('steps',       'std'),
    mean_time    = ('time_ms',     'mean'),
    std_time     = ('time_ms',     'std'),
    mean_switch  = ('switch_rate', 'mean'),
    std_switch   = ('switch_rate', 'std')
).reset_index()


In [ ]:
from matplotlib import pyplot as plt

x = summary['radius']

# compute a small offset based on the data range
steps = summary['mean_steps']
time = summary['mean_time']
switch = summary['mean_switch']

offset_steps = 0.05 * (steps.max() - steps.min())
offset_time  = 0.05 * (time.max()  - time.min())
offset_switch= 0.05 * (switch.max()- switch.min())

# Avg Steps Survived vs. Radius (with extra spacing)
plt.figure(figsize=(5, 4))
plt.plot(x, steps, '-o')
for xi, yi in zip(x, steps):
    plt.text(xi, yi + offset_steps, f"{yi:.1f}", ha='center', va='bottom')
plt.xticks(x)
plt.xlabel('Neighborhood Radius')
plt.ylabel('Avg Steps')
plt.title('Steps Survived vs. Neighborhood Radius')
plt.ylim(steps.min() - offset_steps, steps.max() + 2*offset_steps)
plt.tight_layout()
plt.show()

# Avg Decision Time vs. Radius (with extra spacing)
plt.figure(figsize=(5, 4))
plt.plot(x, time, '-s', color='cornflowerblue')
for xi, yi in zip(x, time):
    plt.text(xi, yi + offset_time, f"{yi:.1f}", ha='center', va='bottom')
plt.xticks(x)
plt.xlabel('Neighborhood Radius')
plt.ylabel('Avg Decision Time (ms)')
plt.title('Decision Time vs. Neighborhood Radius')
plt.ylim(time.min() - offset_time, time.max() + 2*offset_time)
plt.tight_layout()
plt.show()

# Controller Stability vs. Radius (with extra spacing)
plt.figure(figsize=(5, 4))
y_switch = summary['mean_switch']
bars = plt.bar(x, y_switch, alpha=0.7)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height, f"{height:.2f}",
             ha='center', va='bottom')
plt.xticks(x)
plt.xlabel('Neighborhood Radius')
plt.ylabel('Avg Action Switch Rate')
plt.title('Controller Stability vs. Neighborhood Radius')
plt.tight_layout()
plt.show()
